In [1]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

# Parameters
observer = (250, 250)  # Row, Column of observer point
azimuth_range = (45, 90)  # Azimuth in degrees
elevation_angle = (5, 15)  # Vertical angle in degrees (optional for 3D)
radius = 500  # Maximum distance

# Create a raster grid
rows, cols = 500, 500
cell_size = 1
x, y = np.meshgrid(np.arange(cols), np.arange(rows))
dx = x - observer[1]
dy = y - observer[0]
r = np.sqrt(dx**2 + dy**2)  # Distance from observer
theta = (np.degrees(np.arctan2(dy, dx)) + 360) % 360  # Azimuth

# Mask cells outside constraints
mask = (azimuth_range[0] <= theta) & (theta <= azimuth_range[1]) & (r <= radius)

# Create raster data
raster_data = np.where(mask, 1, 0)  # 1 for visible, 0 for not visible

# Save to GeoTIFF
transform = from_origin(-250, 250, cell_size, cell_size)  # Example extent
with rasterio.open(
    "visibility_plane.tif",
    "w",
    driver="GTiff",
    height=rows,
    width=cols,
    count=1,
    dtype=raster_data.dtype,
    crs="EPSG:4326",
    transform=transform,
) as dst:
    dst.write(raster_data, 1)
